In [1]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import json
import pandas as pd
import csv
import datetime
from datetime import date, timedelta

In [2]:
stat_url =  "static/data/fl_ev_stations.csv"
stat_df = pd.read_csv(stat_url)
stat_df.columns

Index(['Fuel Type Code', 'Station Name', 'Street Address',
       'Intersection Directions', 'City', 'State', 'ZIP', 'Plus4',
       'Station Phone', 'Status Code', 'Expected Date',
       'Groups With Access Code', 'Access Days Time', 'Cards Accepted',
       'BD Blends', 'NG Fill Type Code', 'NG PSI', 'EV Level1 EVSE Num',
       'EV Level2 EVSE Num', 'EV DC Fast Count', 'EV Other Info', 'EV Network',
       'EV Network Web', 'Geocode Status', 'Latitude', 'Longitude',
       'Date Last Confirmed', 'ID', 'Updated At', 'Owner Type Code',
       'Federal Agency ID', 'Federal Agency Name', 'Open Date',
       'Hydrogen Status Link', 'NG Vehicle Class', 'LPG Primary',
       'E85 Blender Pump', 'EV Connector Types', 'Country',
       'Intersection Directions (French)', 'Access Days Time (French)',
       'BD Blends (French)', 'Groups With Access Code (French)',
       'Hydrogen Is Retail', 'Access Code', 'Access Detail Code',
       'Federal Agency Code', 'Facility Type', 'CNG Dispenser N

In [3]:
#remove unecessary columns
stat_df = stat_df[['Fuel Type Code', 'Station Name', 'Street Address','Latitude', 'Longitude', 'ZIP', 'Open Date']]
stat_df2 = stat_df.dropna()
stat_df2.head()

,Fuel Type Code,Station Name,Street Address,Latitude,Longitude,ZIP,Open Date
6,ELEC,City of Delray Beach - Banker's Row Parking Lot,200 NE 1st Ave,26.465314,-80.072190,33444,6/15/2011
8,ELEC,Firkins Nissan,1611 Cortez Rd W,27.462023,-82.578036,34207,3/15/2011
9,ELEC,Maus Nissan,937 S Suncoast Blvd,28.848733,-82.579470,34448,3/15/2011
10,ELEC,Deland Nissan,2600 S Woodland Blvd,28.983689,-81.300079,32720,3/15/2011
11,ELEC,Delray Nissan,2200 S Federal Hwy,26.435016,-80.072333,33483,3/15/2011


In [4]:
stat_df2 = stat_df2.set_index(['Open Date'])
stat_df2.head()

,Fuel Type Code,Station Name,Street Address,Latitude,Longitude,ZIP
Open Date,,,,,,
6/15/2011,ELEC,City of Delray Beach - Banker's Row Parking Lot,200 NE 1st Ave,26.465314,-80.072190,33444
3/15/2011,ELEC,Firkins Nissan,1611 Cortez Rd W,27.462023,-82.578036,34207
3/15/2011,ELEC,Maus Nissan,937 S Suncoast Blvd,28.848733,-82.579470,34448
3/15/2011,ELEC,Deland Nissan,2600 S Woodland Blvd,28.983689,-81.300079,32720
3/15/2011,ELEC,Delray Nissan,2200 S Federal Hwy,26.435016,-80.072333,33483


In [7]:
stat_date_df = stat_df2.loc['1/1/2018':'12/21/2020']
stat_date_df.head()

,Fuel Type Code,Station Name,Street Address,Latitude,Longitude,ZIP
Open Date,,,,,,
1/1/2018,ELEC,City of Doral - Police Substation Facility,3719 NW 97th Ave,25.807534,-80.353312,33178
2/15/2018,ELEC,Edenson Dental,109 W Bearss Ave,28.088298,-82.460487,33613
11/9/2017,ELEC,Publix #1558,2952 Aventura Blvd,25.960264,-80.141180,33180
12/7/2017,ELEC,Publix #1562,200 Island Way,27.978860,-82.817276,33767
12/14/2017,ELEC,Publix #1559,443 W County Rd 419,28.646284,-81.137250,33768


In [ ]:
# stat_df.to_csv(r'static/data/fl_ev_stations_clean.csv')